# Get the requied data Like CRN, PIN , Demat ID , Password , Bank , Name, from excel Accounts.xlsx

In [ ]:
# Import required libraries
import pandas as pd

# Path to your Excel file (you can also use a URL if the file is hosted online and accessible)
excel_file_path = 'accounts.xlsx'  # Replace with your actual file path

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=0)  # Use sheet_name to specify sheet index or name

# Convert data types for specific columns
df = df.astype({'crn': str,'pin': str,'demat_id': str,'password': str,'Bank': str,'Name': str})

# Display the first few rows
print(df.head(10))



# Code to apply for the IPO 

In [ ]:
import requests
import json
import pandas as pd
import time
from tenacity import retry, stop_after_attempt, wait_fixed

class MeroShare:
    def __init__(self, dmat_id: str = None, password: str = None, crn: str = None, pin: str = None, bank_name: str = None):
        if dmat_id and "-" in dmat_id:
            dmat_parts = dmat_id.split("-")
            self.__dpid = dmat_parts[0][3:8]
            self.__username = dmat_parts[1]
        self.__password = password
        self.__session = requests.Session()
        self.__capital_id = self.get_capital_id()
        self.__auth_token = None
        self.__applicable_issues = None
        self.crn = crn
        self.pin = pin
        self.bank_name = bank_name

    def get_capital_id(self):
        with self.__session as sess:
            headers = {
                "Accept": "application/json, text/plain, */*",
                "Authorization": "null",
                "Connection": "keep-alive",
                "Origin": "https://meroshare.cdsc.com.np",
                "Referer": "https://meroshare.cdsc.com.np/",
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.105 Safari/537.36"
            }
            sess.headers.update(headers)
            response = sess.get("https://webbackend.cdsc.com.np/api/meroShare/capital/")
            if response.status_code == 200:
                capital_data = response.json()
                for capital in capital_data:
                    if capital['code'] == self.__dpid:
                        return capital['id']
            return None
    @retry(stop=stop_after_attempt(3), wait=wait_fixed(2), reraise=True)
    def login(self):
        if not self.__capital_id:
            print("No capital ID found")
            return False

        with self.__session as sess:
            data = {
                "clientId": self.__capital_id,
                "username": self.__username,
                "password": self.__password
            }

            headers = {
                "Accept": "application/json, text/plain, */*",
                "Authorization": "null",
                "Content-Type": "application/json",
                "Origin": "https://meroshare.cdsc.com.np",
                "Referer": "https://meroshare.cdsc.com.np/"
            }
            sess.headers.update(headers)

            try:
                login_response = sess.post(
                    "https://webbackend.cdsc.com.np/api/meroShare/auth/",
                    json=data
                )

                if login_response.status_code == 200:
                    self.__auth_token = login_response.headers.get("Authorization")
                    return True
                return False
            except Exception as e:
                print(f"Login error: {str(e)}")
                return False

    def get_available_shares(self):
        if not self.__auth_token:
            print("Not logged in")
            return None

        with self.__session as sess:
            data = {
                "filterFieldParams": [
                    {
                        "key": "companyIssue.companyISIN.script",
                        "alias": "Scrip",
                    },
                    {
                        "key": "companyIssue.companyISIN.company.name",
                        "alias": "Company Name",
                    },
                    {
                        "key": "companyIssue.assignedToClient.name",
                        "value": "",
                        "alias": "Issue Manager",
                    },
                ],
                "page": 1,
                "size": 10,
                "searchRoleViewConstants": "VIEW_APPLICABLE_SHARE",
                "filterDateParams": [
                    {
                        "key": "minIssueOpenDate",
                        "condition": "",
                        "alias": "",
                        "value": "",
                    },
                    {
                        "key": "maxIssueCloseDate",
                        "condition": "",
                        "alias": "",
                        "value": "",
                    },
                ]
            }

            headers = {
                "Accept": "application/json, text/plain, */*",
                "Authorization": self.__auth_token,
                "Content-Type": "application/json",
                "Origin": "https://meroshare.cdsc.com.np",
                "Referer": "https://meroshare.cdsc.com.np/"
            }
            sess.headers.update(headers)

            try:
                response = sess.post(
                    "https://webbackend.cdsc.com.np/api/meroShare/companyShare/applicableIssue/",
                    json=data
                )

                if response.status_code == 200:
                    data = response.json()
                    self.__applicable_issues = data.get('object', [])
                    self.show_share_list(self.__applicable_issues)
                    return self.__applicable_issues
                return None
            except Exception as e:
                print(f"Error getting issues: {str(e)}")
                return None

    def show_share_list(self, shares):
        if not shares:
            print("\nNo IPOs currently available.")
            return

        print("\nAvailable Shares:")
        print("-" * 100)
        print("Script    Company Name                    Type        Open Date    Close Date")
        print("-" * 100)

        for share in shares:
            name = share.get('companyName', 'N/A')
            script = share.get('scrip', 'N/A')
            type = share.get('shareTypeName', 'N/A')
            open_date = share.get('issueOpenDate', 'N/A').split(' ')[0:2]
            close_date = share.get('issueCloseDate', 'N/A').split(' ')[0:2]

            print(f"{script:<9} {name[:30]:<30} {type:<11} {' '.join(open_date):<12} {' '.join(close_date)}")
        print("-" * 100)

    def apply_for_share(self, script_code: str, qty: int):
        print("\nStarting share application process...")

        with self.__session as sess:
            try:
                # Get applicable issues if not already fetched
                if not self.__applicable_issues:
                    self.__applicable_issues = self.get_available_shares()

                # Find the matching issue
                issue_to_apply = None
                for issue in self.__applicable_issues:
                    if str(issue.get("scrip")) == script_code:
                        issue_to_apply = issue
                        break

                if not issue_to_apply:
                    print("Provided Script doesn't match any of the applicable issues!")
                    return False

                share_id = issue_to_apply.get('companyShareId')

                if issue_to_apply.get("action"):
                    status = issue_to_apply.get("action")
                    print(f"Couldn't apply for issue! - {status}")
                    return False

                headers = {
                    "Accept": "application/json, text/plain, */*",
                    "Authorization": self.__auth_token,
                    "Content-Type": "application/json",
                    "Origin": "https://meroshare.cdsc.com.np",
                    "Referer": "https://meroshare.cdsc.com.np/"
                }
                sess.headers.update(headers)

                # Get bank details with improved matching
                bank_req = sess.get("https://webbackend.cdsc.com.np/api/meroShare/bank/").json()

                bank_id = None
                input_bank = self.bank_name.lower().strip()

                # More flexible bank name matching
                for bank_ in bank_req:
                    if (input_bank in bank_['name'].lower() or
                        bank_['name'].lower() in input_bank):
                        bank_id = bank_["id"]
                        break

                if bank_id is None:
                    print(f"Bank '{self.bank_name}' not found. Available banks:")
                    for bank_ in bank_req:
                        print(f"- {bank_['name']}")
                    return False

                bank_specific_req = sess.get(
                    f"https://webbackend.cdsc.com.np/api/meroShare/bank/{bank_id}"
                )
                bank_details = bank_specific_req.json()

                if isinstance(bank_details, list) and len(bank_details) > 0:
                    bank_details = bank_details[0]

                # Prepare application data
                data = json.dumps({
                    "accountBranchId": bank_details.get("accountBranchId"),
                    "accountNumber": bank_details.get("accountNumber"),
                    "accountTypeId": bank_details.get("accountTypeId"),
                    "appliedKitta": qty,
                    "bankId": bank_id,
                    "boid": self.__username,
                    "customerId": bank_details.get("id"),
                    "crnNumber": self.crn,
                    "companyShareId": share_id,
                    "demat": f"130{self.__dpid}{self.__username}",
                    "transactionPIN": self.pin
                })

                apply_req = sess.post(
                    "https://webbackend.cdsc.com.np/api/meroShare/applicantForm/share/apply",
                    data=data
                )

                if apply_req.status_code == 201:
                    print(f"Successfully applied for {qty} Kitta!")
                    return True
                else:
                    response_data = apply_req.json()
                    print(f"Application failed! Response: {response_data}")
                    return False

            except Exception as error:
                print(f"Error during application: {str(error)}")
                return False

def get_ipo_list(first_account):
    """Get IPO list using first account credentials"""
    ms = MeroShare(
        dmat_id=first_account['demat_id'],
        password=first_account['password'],
        crn=first_account['crn'],
        pin=first_account['pin'],
        bank_name=first_account['Bank']
    )

    if ms.login():
        return ms.get_available_shares()
    return None

def batch_apply_shares(df: pd.DataFrame, script_code: str, quantity: int) -> pd.DataFrame:
    results = []

    for _, row in df.iterrows():
        ms = MeroShare(
            dmat_id=row['demat_id'],
            password=row['password'],
            crn=row['crn'],
            pin=row['pin'],
            bank_name=row['Bank']
        )

        print(f"\nProcessing for: {row['Name']}")

        if ms.login():
            success = ms.apply_for_share(script_code, quantity)
            status = 'Success' if success else 'Failed'
        else:
            status = 'Login Failed'

        results.append({
            'Name': row['Name'],
            'Demat': row['demat_id'],
            'Status': status,
            'Script': script_code,
            'Quantity': quantity
        })

        time.sleep(1)  # Delay between requests

    return pd.DataFrame(results)

# Main execution
if __name__ == "__main__":
    # Read the Excel file containing account details
    try:
        #df = pd.read_excel('accounts.xlsx')

        # Get first account for checking available shares
        first_account = df.iloc[0]
        available_shares = get_ipo_list(first_account)

        if available_shares:
            script_code = input("Enter script code: ")
            quantity = int(input("Enter quantity to apply: "))

            results_df = batch_apply_shares(df, script_code, quantity)
            print("\nApplication Results:")
            print(results_df)

            # Save results
            results_df.to_excel(f'share_application_results_{script_code}.xlsx', index=False)
        else:
            print("Could not fetch IPO list")

    except Exception as e:
        print(f"Error: {str(e)}")